In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time
import requests
import os
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from pathlib import Path
from datetime import datetime
import sqlite3

In [2]:
from splinter import Browser
from selenium.webdriver.chrome.service import Service

my_service = Service(executable_path=r'C:\Users\pc\OneDrive\Desktop\TCG-scraper\chromedriver.exe')
browser = Browser('chrome', service=my_service)

In [3]:
dbtable ="MID"
url_string= "innistrad-midnight-hunt"

In [4]:
# visit browser and make soup
url =f"https://shop.tcgplayer.com/price-guide/magic/{url_string}"
browser.visit(url)
browser.driver.maximize_window()

In [5]:
time.sleep(5)

In [6]:
req = browser.html

soup = BeautifulSoup(req, "html")

In [7]:
names= []
title_tags = soup.find_all('a', class_ = "pdp-url")
#title_tags
for i in title_tags:
   names.append(i.text)

In [8]:
prices =[]
price = soup.find_all('td', class_ = "tcg-table-body__cell tcg-table-body__cell--align-right")
#price
count = -1
for i in price:
    count = count +1
    if  count%2 > 0 :
        prices.append(i.text)

In [9]:
# turn lists into new data frame
price_list= zip(names, prices)
mkt_df = pd.DataFrame(price_list,columns = ['Name', 'Market Price']) 
mkt_df
#mkt_df.columns = ['Name', 'Market Price']
mkt_df.head(3)

,Name,Market Price
0,Abandon the Post,$0.02
1,"Adeline, Resplendent Cathar",$6.97
2,"Adeline, Resplendent Cathar (Showcase)",$5.87


In [10]:
mkt_df['date']= datetime.today().strftime('%Y-%m-%d')
mkt_df.head(3)

,Name,Market Price,date
0,Abandon the Post,$0.02,2023-11-21
1,"Adeline, Resplendent Cathar",$6.97,2023-11-21
2,"Adeline, Resplendent Cathar (Showcase)",$5.87,2023-11-21


In [11]:
file_path= ("../Resources/inventory.db")

In [12]:
price =  Path("../Resources/inventory.db")
con = sqlite3.connect(price)
cur = con.cursor()

In [13]:
q = cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in q.fetchall():
    print(name[0])

10E
BRO
DGM
DMU
ELD
GTC
MAT
MID
MOM
NEO
ONE
RTR
SNC
VOW
WOE


In [14]:
db_df = pd.read_sql_query(f"SELECT * FROM '{dbtable}'", con)

db_df["MktPrice"] = 0
db_df.head(1)

,setCode,name,rarity,manaCost,colorIdentity,types,subtypes,power,toughness,quantity,nm,lp,p,foil,text,MktPrice
0,MID,Consuming Blob,mythic,{3}{G}{G},G,Creature,Ooze,0,0,0,0,0,0,0,Consuming Blob's power is equal to the number ...,0


In [15]:
cur.execute(f"ALTER TABLE {dbtable} ADD COLUMN  MktPrice VARCHAR(10)")

In [16]:
# Match inventory df  to scrapped data and append current Market price to inventory df
count = 0
while count < len(db_df): # next itteration if loop fails 
    try:
        for i in range(count , len(db_df)):
            search = db_df.loc[i,['name']]   # 
            search = search['name']
            search_df = mkt_df.loc[mkt_df['Name'] == search]
                           # break if no match
            search_df.reset_index(drop=True, inplace=True)
            search_result = search_df["Market Price"][0]
            db_df.loc[i,['MktPrice']] = search_result
            count = count + 1
    except: # if no match 
            db_df.loc[i,['MktPrice']] = "na"
            count = count + 1

In [17]:
                        # change db table mtkPrice to set_df["Mktprice"]

for i in range(len(mkt_df)):
    search = mkt_df.loc[i,['Name']]   # 
    search = search['Name']
    x = mkt_df.loc[i,['Market Price']]  
    x = x['Market Price']
    insert_sql = f'''UPDATE  {dbtable}  set MktPrice ="{x}" where "name" = "{search}"'''
    cur.execute(insert_sql)
    con.commit()

In [20]:
set_df = pd.read_sql_query(f"SELECT * FROM {dbtable}", con)
set_df.head(280)

,setCode,name,rarity,manaCost,colorIdentity,types,subtypes,power,toughness,quantity,nm,lp,p,foil,text,MktPrice
0,MID,Consuming Blob,mythic,{3}{G}{G},G,Creature,Ooze,0,0,0,0,0,0,0,Consuming Blob's power is equal to the number ...,$0.33
1,MID,Old Stickfingers,rare,{X}{B}{G},"B, G",Creature,Horror,0,0,0,0,0,0,0,"When you cast this spell, reveal cards from th...",$0.31
2,MID,Morkrut Behemoth,common,{4}{B},B,Creature,"Zombie, Giant",7,6,1,0,1,0,0,"As an additional cost to cast this spell, sacr...",$0.01
3,MID,Burly Breaker // Dire-Strain Demolisher,uncommon,{3}{G}{G},G,Creature,"Human, Werewolf",6,5,2,0,1,1,1,Ward {1} (Whenever this creature becomes the t...,None
4,MID,Dreadhound,uncommon,{4}{B}{B},B,Creature,"Demon, Dog",6,6,0,0,0,0,0,"When Dreadhound enters the battlefield, mill t...",$0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,MID,Vanquish the Horde,rare,{6}{W}{W},W,Sorcery,0,0,0,0,0,0,0,0,This spell costs {1} less to cast for each cre...,$3.60
276,MID,Vivisection,uncommon,{3}{U},U,Sorcery,0,0,0,0,0,0,0,0,"As an additional cost to cast this spell, sacr...",$0.01
277,MID,Wake to Slaughter,rare,{3}{B}{R},"B, R",Sorcery,0,0,0,0,0,0,0,0,Choose up to two target creature cards in your...,$0.06
278,MID,Winterthorn Blessing,uncommon,{G}{U},"G, U",Sorcery,0,0,0,0,0,0,0,0,Put a +1/+1 counter on up to one target creatu...,$0.05
